# External Data Retrieval
Uses external APIs for more data. Saves data for further analysis and offline use.

In [3]:
# All imports
import pandas as pd
import pickle
from pathlib import Path
import utils

In [6]:
# Ensembl Protein ID to Sequence mapping
train_ensp_sequence_map: dict[str, str] = {}
test_ensp_sequence_map: dict[str, str] = {}

train_ensp_series: pd.Series = pd.read_csv(utils.RAW_TRAIN_PATH, usecols=["ensp"], dtype={"ensp": str})["ensp"]
test_ensp_series: pd.Series = pd.read_csv(utils.RAW_TEST_PATH, usecols=["ensp"], dtype={"ensp": str})["ensp"]

train_sequence_series: pd.Series = train_ensp_series.apply(lambda ensp: utils.get_full_sequence(ensp, train_ensp_sequence_map))
test_sequence_series: pd.Series = test_ensp_series.apply(lambda ensp: utils.get_full_sequence(ensp, test_ensp_sequence_map))

# Save the mappings using pickle
# TODO: save pickle files to utils.TRAIN_ENSP_SEQUENCE_MAP_PATH and utils.TEST_ENSP_SEQUENCE_MAP_PATH
with open("data/test/test_pickle.pkl", "wb") as f:
    pickle.dump(test_ensp_sequence_map)

with open("data/test/train_pickle.pkl", "wb") as f:
    pickle.dump(train_ensp_sequence_map)

ReadTimeout: HTTPSConnectionPool(host='rest.ensembl.org', port=443): Read timed out. (read timeout=3)

In [ ]:
# TODO: retrieve more data and save to pickle files